In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

> This case study will help us to understand the These data are the results of a chemical analysis of wines grown in the same region in Italy but derived from three different cultivars. The analysis determined the quantities of 13 constituents found in each of the three types of wines. We will focus on the following stages namely -


* Data Preparation
* Feature Engineering
* Feature Subset Selection
* Model Training
* Model Evaluation

**Dataset Data**:

https://www.kaggle.com/sadeghjalalian/wine-customer-segmentation

**Possible Work to be done** :

* Write a Data Science Proposal for achieving the objective mentioned.
* Perform exploratory analysis on the data.
* Perform data wrangling / pre-processing.
* Apply any 2 features engineering technique.
* Plot top 10 features.
* Identification of the performance parameters to be improved, for the given problem statement.
* Design Machine Learning models – Logistic regression and Decision tree to predict.
* Compare the performance of selected feature engineering techniques.
* Compare the performance of the 2 classifiers – Logistic regression and Decision tree to predict.
* 10.Conclusions.

**Package Imports**

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.tree import DecisionTreeClassifier # Import Decision Tree Classifier
from sklearn.model_selection import train_test_split # Import train_test_split function
from sklearn.linear_model import LogisticRegression # to apply the Logistic regression
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
from sklearn.feature_selection import chi2
from sklearn.feature_selection import mutual_info_classif
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, roc_curve, roc_auc_score, classification_report
from sklearn.model_selection import GridSearchCV
from statsmodels.stats.outliers_influence import variance_inflation_factor 
from statsmodels.tools.tools import add_constant
import warnings
warnings.filterwarnings("ignore")
import sklearn.metrics as metrics
from sklearn.metrics import auc,roc_curve,f1_score,recall_score,precision_score
from sklearn.preprocessing import StandardScaler

> Reading data from Wine Datasets

In [ ]:
data =pd.read_csv(os.path.join(dirname, filename))

**Confirm the imports**

In [ ]:
data.head()

In [ ]:
data.shape

In [ ]:
data.info()

All the columns are numeric in nature

In [ ]:
data.describe()

# Data Preparation

Checking for unique data values

In [ ]:
def show_unique_values(data_frame):
    print("Unique value for dataset attributes :\n")
    for column in data_frame.columns:
        print(column, " " ,data_frame[column].unique(), "\n")   

In [ ]:
show_unique_values(data)

**Missing Values**
> Lets see how the missing values can be replaced in the dataset. First check whereall the missing values are present.
> 
> Take a closer look at the actual missing value count. 'False' means cell has a value whereas 'True" means cell is missing value. Output the count for different attributes of dataframe.

In [ ]:
def show_missing_values(data):
    missing_data = data.isnull()
    for column in missing_data.columns.values.tolist():
        print(column)
        print (missing_data[column].value_counts())
        print("")

In [ ]:
show_missing_values(data)

In [ ]:
data.isnull().sum()

None of the values is missing.

**Duplicate Records Analysis**

In [ ]:
#Verifing duplicate records in wine datasets
duplicate = data[data.duplicated()] 
duplicate 


> There is no duplicate Data

**Noisy Data Verification**

In [ ]:
sns.set(style="whitegrid")
data.iloc[:,0:13].boxplot(figsize=(20,8))

In [ ]:
#box plot
sns.set(style="whitegrid")
sns.set(rc={'figure.figsize':(4,2)})
sns.boxplot(x=data['Ash'])
plt.show()
sns.boxplot(x=data['Ash_Alcanity'])
plt.show()
sns.boxplot(x=data['Magnesium'])
plt.show()
sns.boxplot(x=data['Proanthocyanins'])
plt.show()

**There some outliers in the following features.**
 
* Ash
* Ash_Alcanity
* Magnesium
* Proanthocyanins

**Verify Skewness in data**
* Skewness is a measure of asymmetry of a distribution.
* 
* If the skewness is between -0.5 and 0.5, the data are fairly symmetrical
* If the skewness is between -1 and — 0.5 or between 0.5 and 1, the data are moderately skewed
* If the skewness is less than -1 or greater than 1, the data are highly skewed

In [ ]:
# skewness along the index axis 
data.skew(axis = 0, skipna = True)

The features:

Magnesium

Malic_Acid
are highly skewed and hence should be transformed

**Verify target class imbalance**

In [ ]:
data['Customer_Segment'].value_counts(sort = False, normalize = True)*100

> Customer Segment looks like balanced with 3 different types. No imabalance treatment required

In [ ]:
X = data.iloc[:,0:13]  #independent columns
y = data.iloc[:,-1]    #target column i.e price range

**Feature Subset Selection**

In [ ]:
def show_top_univariate_filters(data, score_func, top_k):
    X = data.iloc[:,0:13]  #independent columns
    y = data.iloc[:,-1]    #target column i.e price range

    if score_func == "chi2":
        func = chi2
    elif score_func == "f_classif":
        func = f_classif
    elif score_func == "mutual_info_classif":
        func = mutual_info_classif
    
    #apply SelectKBest class to extract top k best features
    bestfeatures = SelectKBest(score_func=func, k=top_k)
    fit = bestfeatures.fit(X,y)

    dfscores = pd.DataFrame(fit.scores_)
    dfcolumns = pd.DataFrame(X.columns)

    #concat two dataframes for better visualization 
    featureScores = pd.concat([dfcolumns,dfscores],axis=1)
    featureScores.columns = ['features','Score']  #naming the dataframe columns
    print(featureScores.nlargest(top_k,'Score'))  #print 10 best features

In [ ]:
show_top_univariate_filters(data, 'chi2', 10)

In [ ]:
show_top_univariate_filters(data, 'f_classif', 10)

**Correlation Matrix with Heatmap**

In [ ]:
#get correlations of each features in dataset
corrmat = data.corr()
top_corr_features = corrmat.index

#plot heat map
plt.figure(figsize=(20,20))
sns.heatmap(data[top_corr_features].corr(), annot=True, cmap=plt.cm.Reds)
plt.show()

> Few observations : Following features are correlated when correlation >0.7

* Total_Phenols and Flavanoids
* Total_Phenols and OD280
* Flavanoids and OD280

**Variance inflation factor**

* Variance inflation factor (VIF) is a measure of the amount of multicollinearity in a set of multiple regression variables

In [ ]:
VIF = pd.Series([variance_inflation_factor(X.values, i) 
               for i in range(X.shape[1])], 
              index=X.columns).to_frame()


VIF.rename(columns={VIF.columns[0]: 'VIF'},inplace = True)
VIF[~VIF.isin([np.nan, np.inf, -np.inf]).any(1)]

# Feature Engineering

In [ ]:
# dropping Flavanoids column
X.drop("Flavanoids", axis=1, inplace=True)

**Transform skewed entities**

In [ ]:
X['Magnesium'] = np.log(X['Magnesium'])
X['Malic_Acid'] = np.log(X['Malic_Acid'])

In [ ]:
X.skew(axis = 0, skipna = True) 

# Feature Subset Selection

In [ ]:
def show_top_decition_classifier_feature(data, classifier, top_k):

    if classifier == "ExtraTreesClassifier":
        classifier = ExtraTreesClassifier
    elif classifier == "DecisionTreeClassifier":
        classifier = DecisionTreeClassifier
       
    model = classifier()
    model.fit(X,y)

    #use inbuilt class feature_importances of tree based classifiers
    print(model.feature_importances_) 

    #plot graph of feature importances for better visualization
    feat_importances = pd.Series(model.feature_importances_, index=X.columns)
    feat_importances.nlargest(top_k).plot(kind='barh')
    plt.show()

In [ ]:
show_top_decition_classifier_feature(data, "DecisionTreeClassifier", 10)

In [ ]:
show_top_decition_classifier_feature(data, "ExtraTreesClassifier", 10)

# Model Training

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1) # 70% training and 30% test

**Logistic regression**

In [ ]:
logistic = LogisticRegression()
logistic_fit=logistic.fit(X_train,y_train)
y_pred_test=logistic_fit.predict(X_test)

In [ ]:
print('Logistic model  Test Accuracy: {:.2f}%'.format(accuracy_score(y_test, y_pred_test) * 100))

print('Logistic model Classification report:\n\n', classification_report(y_test, y_pred_test))

print('Logistic model Training set score: {:.2f}%'.format(logistic_fit.score(X_train, y_train) * 100))

In [ ]:
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_test, y_pred_test)

print('Confusion matrix\n\n', cm)

print('\nTrue Positives(TP) = ', cm[0,0])

print('\nTrue Negatives(TN) = ', cm[1,1])

print('\nFalse Positives(FP) = ', cm[0,1])

print('\nFalse Negatives(FN) = ', cm[1,0])

In [ ]:
# visualize confusion matrix with seaborn heatmap

cm_matrix = pd.DataFrame(data=cm)

sns.heatmap(cm_matrix, annot=True, fmt='d', cmap='YlGnBu')

**Decision tree**

In [ ]:
# Create Decision Tree classifer object
clf = DecisionTreeClassifier()

# Train Decision Tree Classifer
clf = clf.fit(X_train,y_train)

#Predict the response for test dataset
y_pred = clf.predict(X_test)

In [ ]:
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

In [ ]:
# Create Decision Tree classifer object
clf = DecisionTreeClassifier(criterion="entropy", max_depth=5)

# Train Decision Tree Classifer
clf = clf.fit(X_train,y_train)

#Predict the response for test dataset
y_pred = clf.predict(X_test)


In [ ]:
print('Test Accuracy:',accuracy_score(y_test,y_pred))

In [ ]:
print('Train Accuracy:',accuracy_score(y_train,clf.predict(X_train)))

**Hyper-parameter Tuning**

In [ ]:
param_grid = {
'criterion':['gini','entropy'],
'max_depth':[4,6,8,12]
}

In [ ]:
g_dt = GridSearchCV(clf,param_grid=param_grid,n_jobs=-1,cv=5,scoring='accuracy')

In [ ]:
g_dt.fit(X_train,y_train)

In [ ]:
g_dt.best_params_

In [ ]:
g_dt.best_score_

In [ ]:
f_dt = DecisionTreeClassifier(criterion='gini',max_depth=12)
f_dt.fit(X_train,y_train)
y_pred = f_dt.predict(X_test)
print('Test Accuracy:',accuracy_score(y_test,y_pred))

In [ ]:
print('Train Accuracy:',accuracy_score(y_train,f_dt.predict(X_train)))

In [ ]:
cm = confusion_matrix(y_test,y_pred)
cm
print(classification_report(y_test,y_pred))

In [ ]:
pred_prob1 = logistic_fit.predict_proba(X_test)
pred_prob2 = f_dt.predict_proba(X_test)

In [ ]:
# roc curve for models
fpr1, tpr1, thresh1 = roc_curve(y_test, pred_prob1[:,1], pos_label=1)
fpr2, tpr2, thresh2 = roc_curve(y_test, pred_prob2[:,1], pos_label=1)

# roc curve for tpr = fpr 
random_probs = [0 for i in range(len(y_test))]
p_fpr, p_tpr, _ = roc_curve(y_test, random_probs, pos_label=1)

In [ ]:
plt.style.use('seaborn')

# plot roc curves
plt.plot(fpr1, tpr1, linestyle='--',color='orange', label='Logistic Regression')
plt.plot(fpr2, tpr2, linestyle='--',color='green', label='Decision tree')
plt.plot(p_fpr, p_tpr, linestyle='--', color='blue')
# title
plt.title('ROC curve')
# x label
plt.xlabel('False Positive Rate')
# y label
plt.ylabel('True Positive rate')

plt.legend(loc='best')
plt.savefig('ROC',dpi=300)
plt.show();

In [ ]:
algo_list=[]
algo_list=["Logistic Regression","Decision Tree"]

In [ ]:
score_list=[]
score_list.append(accuracy_score(y_test, y_pred_test) * 100)
score_list.append(accuracy_score(y_test, y_pred) * 100)

In [ ]:

precision=[]
precision.append(precision_score(y_test, y_pred_test, average='macro'))
precision.append(precision_score(y_test, y_pred, average='macro'))

In [ ]:

recall=[]
recall.append(recall_score(y_test, y_pred_test, average='macro'))
recall.append(recall_score(y_test, y_pred, average='macro'))

In [ ]:

f1score=[]
f1score.append(f1_score(y_test, y_pred_test, average='macro'))
f1score.append(f1_score(y_test, y_pred, average='macro'))

In [ ]:
score={"Algorithims":algo_list,"Scores":score_list,"precision_score":precision,"recall_score":recall,"f1_score":f1score}
score

In [ ]:
df=pd.DataFrame(score)
df